In [2]:
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import sklearn

In [3]:
# Enter your code here. Read input from STDIN. Print output to STDOUT
import os
import numpy as np
import pandas as pd
import sklearn

from sklearn.model_selection import KFold
from sklearn.svm import SVC
#from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score


trainpath = "./train.csv"
testpath = "./test.csv"
predpath = "./prediction.csv"

train_df = pd.read_csv(trainpath, header = None)
test_df = pd.read_csv(testpath, header = None)

data_train = np.array(train_df)[:, :294]
label_train = np.array(train_df)[:, 294:]
data_test = np.array(test_df)[:, :294]
label_test = np.zeros((test_df.shape[0], 6))

num_labels = label_train.shape[1]
kf = KFold(n_splits=5, shuffle=True, random_state=42)

models = []
for i in range(num_labels):
    params = {
        'C': 1.0,
        'kernel': 'rbf',
        'gamma': 'scale',
        'random_state': 42
    }
    preds = []
    scores = []
    models.append([])
    for train_index, test_index in kf.split(data_train):
        X_train, X_test = data_train[train_index], data_train[test_index]
        y_train, y_test = label_train[train_index, i], label_train[test_index, i]
        model = SVC(**params)
        #model = LogisticRegression(random_state=42)
        model.fit(X_train, y_train)
        models[i].append(model)
        #y_pred = model.predict(X_test)
        #score = accuracy_score(y_test, y_pred)
        #scores.append(score)
        pred = model.predict(data_test)
        preds.append(pred)
    preds = np.array(preds)
    preds = np.mean(preds, axis = 0)
    preds = np.where(preds>0.5, 1, 0)
    label_test[:, i] = preds
    #print(preds.shape)
    #print('label%d交叉验证结果：'%(i), scores)
    #print('label%d平均准确率：'%(i), sum(scores) / len(scores))
    
output = pd.DataFrame(label_test, dtype = int)

output.to_csv(predpath, header=False)